#### WPC

In [1]:
import matplotlib.pyplot as plt
import numpy as np
# import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from warnings import filterwarnings
import tensorflow as tf
from tensorflow import keras

# import warnings
# warnings.filterwarnings('ignore')
# sns.set(style = 'whitegrid')
# from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
# import plotly.express as px
# import seaborn as sns

# Modelling Libraries
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from keras.wrappers.scikit_learn import KerasClassifier

#accurcy
from sklearn.metrics import precision_score,accuracy_score
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold

%matplotlib inline

In [2]:
df = pd.read_csv('../data/water_potability.csv')

df.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,NaN,204.890455,20791.318981,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0
1,3.716080,129.422921,18630.057858,6.635246,NaN,592.885359,15.180013,56.329076,4.500656,0
2,8.099124,224.236259,19909.541732,9.275884,NaN,418.606213,16.868637,66.420093,3.055934,0
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3276 entries, 0 to 3275
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ph               2785 non-null   float64
 1   Hardness         3276 non-null   float64
 2   Solids           3276 non-null   float64
 3   Chloramines      3276 non-null   float64
 4   Sulfate          2495 non-null   float64
 5   Conductivity     3276 non-null   float64
 6   Organic_carbon   3276 non-null   float64
 7   Trihalomethanes  3114 non-null   float64
 8   Turbidity        3276 non-null   float64
 9   Potability       3276 non-null   int64  
dtypes: float64(9), int64(1)
memory usage: 256.1 KB


In [4]:
df.isnull().sum()

ph                 491
Hardness             0
Solids               0
Chloramines          0
Sulfate            781
Conductivity         0
Organic_carbon       0
Trihalomethanes    162
Turbidity            0
Potability           0
dtype: int64

In [5]:
print(df.shape)
df.describe()

(3276, 10)


,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
count,2785.000000,3276.000000,3276.000000,3276.000000,2495.000000,3276.000000,3276.000000,3114.000000,3276.000000,3276.000000
mean,7.080795,196.369496,22014.092526,7.122277,333.775777,426.205111,14.284970,66.396293,3.966786,0.390110
std,1.594320,32.879761,8768.570828,1.583085,41.416840,80.824064,3.308162,16.175008,0.780382,0.487849
min,0.000000,47.432000,320.942611,0.352000,129.000000,181.483754,2.200000,0.738000,1.450000,0.000000
25%,6.093092,176.850538,15666.690297,6.127421,307.699498,365.734414,12.065801,55.844536,3.439711,0.000000
50%,7.036752,196.967627,20927.833607,7.130299,333.073546,421.884968,14.218338,66.622485,3.955028,0.000000
75%,8.062066,216.667456,27332.762127,8.114887,359.950170,481.792304,16.557652,77.337473,4.500320,1.000000
max,14.000000,323.124000,61227.196008,13.127000,481.030642,753.342620,28.300000,124.000000,6.739000,1.000000


In [6]:
df['ph'] = df['ph'].replace(np.nan, df['ph'].median())
df['Sulfate'] = df['Sulfate'].replace(np.nan, df['Sulfate'].median())
df['Trihalomethanes'] = df['Trihalomethanes'].replace(np.nan, df['Trihalomethanes'].median())

df.isnull().sum()

ph                 0
Hardness           0
Solids             0
Chloramines        0
Sulfate            0
Conductivity       0
Organic_carbon     0
Trihalomethanes    0
Turbidity          0
Potability         0
dtype: int64

In [7]:
print(df.shape)
df.describe()

(3276, 10)


,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
count,3276.000000,3276.000000,3276.000000,3276.000000,3276.000000,3276.000000,3276.000000,3276.000000,3276.000000,3276.000000
mean,7.074194,196.369496,22014.092526,7.122277,333.608364,426.205111,14.284970,66.407478,3.966786,0.390110
std,1.470040,32.879761,8768.570828,1.583085,36.143851,80.824064,3.308162,15.769958,0.780382,0.487849
min,0.000000,47.432000,320.942611,0.352000,129.000000,181.483754,2.200000,0.738000,1.450000,0.000000
25%,6.277673,176.850538,15666.690297,6.127421,317.094638,365.734414,12.065801,56.647656,3.439711,0.000000
50%,7.036752,196.967627,20927.833607,7.130299,333.073546,421.884968,14.218338,66.622485,3.955028,0.000000
75%,7.870050,216.667456,27332.762127,8.114887,350.385756,481.792304,16.557652,76.666609,4.500320,1.000000
max,14.000000,323.124000,61227.196008,13.127000,481.030642,753.342620,28.300000,124.000000,6.739000,1.000000


In [8]:
X = df.drop(['Potability'], axis = 1).values
y = df['Potability'].values

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [10]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
filterwarnings('ignore')

models = [
    ('SVM', LinearSVC()),
    ('KNN',KNeighborsClassifier(n_neighbors=10)),
    ('MLP',MLPClassifier(max_iter=100))
    ]

results = []
names = []
finalResults = []

for name,model in models:
    model.fit(X_train, y_train)
    model_results = model.predict(X_test)
    score = precision_score(y_test, model_results,average='macro')
    results.append(score)
    names.append(name)
    finalResults.append((name,score))
    
finalResults.sort(key=lambda k:k[1],reverse=True)

In [12]:
finalResults

[('MLP', 0.6501489165040567),
 ('KNN', 0.610276073619632),
 ('SVM', 0.30671414038657174)]

In [13]:
def build_clf(unit, optimizer, rate, activation):
# def build_clf(unit, activation):
    # creating the layers of the NN
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=unit, activation=activation))
    model.add(tf.keras.layers.Dense(units=unit, activation=activation))
    model.add(tf.keras.layers.Dense(units=unit, activation=activation))
    model.add(tf.keras.layers.Dense(units=unit, activation=activation))
    model.add(tf.keras.layers.Dense(units=unit, activation=activation))
    model.add(keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) #여기 activation은 어케 처리하지?
    model.add(keras.layers.Dropout(rate=rate))
    # model.add(keras.layers.Dropout(rate=0.1))
    model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy']) #Label이 0 또는 1이기 때문에 Binary
    # model.compile(optimizer = "Adam", loss = 'binary_crossentropy', metrics = ['accuracy']) #Label이 0 또는 1이기 때문에 Binary
    return model

In [14]:
model_params = {
    'SVM':
        {
            'model':LinearSVC(),
            'params':
                {
                    'C': [1, 10, 100, 1000],
                    'loss': ['hinge', 'squared_hinge']
                }
        },
    'KNN':
        {
            'model':KNeighborsClassifier(),
            'params':
                {
                    'n_neighbors' : list(range(1,20)),
                    'weights' : ["uniform", "distance"],
                    'metric' : ['euclidean', 'manhattan', 'minkowski']
                }
        },
    'MLP':
        {
            'model':MLPClassifier(),
            'params':
                {
                    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
                    'activation': ['tanh', 'relu'],
                    'solver': ['sgd', 'adam'],
                    'alpha': [0.0001, 0.05],
                    'learning_rate': ['constant','adaptive'],
                }
        },
    'DMLP':
        {
            'model': KerasClassifier(build_fn=build_clf),
            'params':
                {
                    'batch_size':[100, 20, 50, 25, 32],
		            'nb_epoch':[200, 100, 300, 400],
		            'unit':[5, 6, 10, 11, 12, 15],
                    'optimizer': ['SGD','RMSprop','Adam','Adadelta','Adamax','Nadam','Ftrl'],
                    'rate': [0.1, 0.01, 0.001],
                    'activation': ['softmax','elu','selu','softsign','softplus','relu','tanh','sigmoid','hard_sigmoid','exponential','linear']
                }
        }
}

In [15]:
model_params['DMLP']['model']

In [16]:
# scores = []
# rs = GridSearchCV(model_params['DMLP']['model'], model_params['DMLP']['params'], refit=True, verbose=2)
# rs.fit(X,y)
# scores.append([dict(rs.best_params_),rs.best_score_])
# Tdata=pd.DataFrame(scores,columns=['Parameters','Score'])
# Tdata

Fitting 5 folds for each of 27720 candidates, totalling 138600 fits


2022-06-05 15:20:48.215898: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-05 15:20:48.257534: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-05 15:20:48.257669: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-05 15:20:48.258344: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

7/7 [==============================] - 0s 748us/step - loss: 0.6680 - accuracy: 0.7561


2022-06-05 15:20:49.961160: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


[CV] END activation=softmax, batch_size=100, nb_epoch=200, optimizer=SGD, rate=0.1, unit=5; total time=   2.0s
7/7 [==============================] - 0s 895us/step - loss: 0.6936 - accuracy: 0.4885
[CV] END activation=softmax, batch_size=100, nb_epoch=200, optimizer=SGD, rate=0.1, unit=5; total time=   0.4s
7/7 [==============================] - 0s 707us/step - loss: 0.6579 - accuracy: 0.6397
[CV] END activation=softmax, batch_size=100, nb_epoch=200, optimizer=SGD, rate=0.1, unit=5; total time=   0.4s
7/7 [==============================] - 0s 903us/step - loss: 0.7244 - accuracy: 0.3725
[CV] END activation=softmax, batch_size=100, nb_epoch=200, optimizer=SGD, rate=0.1, unit=5; total time=   0.4s
7/7 [==============================] - 0s 736us/step - loss: 0.7071 - accuracy: 0.5145
[CV] END activation=softmax, batch_size=100, nb_epoch=200, optimizer=SGD, rate=0.1, unit=5; total time=   0.4s
7/7 [==============================] - 0s 710us/step - loss: 0.6376 - accuracy: 0.7561
[CV] END a

KeyboardInterrupt: 

In [ ]:
scores = []
for model_name, params in model_params.items():
    rs = GridSearchCV(params['model'], params['params'], refit=True, verbose=2)
    rs.fit(X,y)
    scores.append([model_name,dict(rs.best_params_),rs.best_score_])
    
Tdata=pd.DataFrame(scores,columns=['Model','Parameters','Score'])
# 자르지 않고 Data frame 표시
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
Tdata

In [ ]:
param = Tdata['Parameters']

models = [
    ('SVM', LinearSVC(**param[0])),
    ('KNN', KNeighborsClassifier(**param[1])),
    ('MLP', MLPClassifier(**param[2])),
    ('DMLP', KerasClassifier(build_fn=build_clf, **param[3]))
    ]

# accuracy=[]
names = []
finalResults = []

#scaler up 
scaler = StandardScaler()

#교차 검증
skf = RepeatedStratifiedKFold(n_splits=5,n_repeats=2)
skf.get_n_splits(X,y)


for name, model in models:
    accuracy=[]
    for train_index, test_index in skf.split(X,y): # 10번 실행
        
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        
        model.fit(X_train,y_train) # 모델 생성
        predictions=model.predict(X_test) # 모델 학습
        score=accuracy_score(y_test,predictions) # 정확도 평가
        accuracy.append(score) #정확도 저장
        names.append(name)
    
    finalResults.append((name,np.mean(accuracy),accuracy))

In [ ]:
pd_models = pd.DataFrame(finalResults,columns=['Model','accuracy_mean','accuracy'])

In [ ]:
pd_models